---
title: "Datasets"
description: This module contains the useful functions to deal with candidates datasets and individual candidate.
---

In [1]:
#| default_exp datasets
#| export
import polars as pl
# import holoviews as hv
import polars.selectors as cs
from loguru import logger
from random import sample
from datetime import timedelta

from pydantic import BaseModel, Field, validate_call
from space_analysis.ds.meta import Meta, PlasmaMeta, TempMeta
from typing import Literal

## Datasets

Fundamental class

In [2]:
#| export
from discontinuitypy.utils.basic import df2ts
from discontinuitypy.utils.plot import plot_candidate as _plot_candidate
from discontinuitypy.integration import combine_features, calc_combined_features
from discontinuitypy.core.pipeline import ids_finder

02-Mar-24 16:35:59: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/discontinuitypy/lib/python3.10/site-packages/pdpipe/__init__.py", line 105, in <module>
    from . import nltk_stages
  File "/Users/zijin/micromamba/envs/discontinuitypy/lib/python3.10/site-packages/pdpipe/nltk_stages.py", line 19, in <module>
    import nltk
ModuleNotFoundError: No module named 'nltk'


02-Mar-24 16:35:59: UserWarning: pdpipe: nltk import failed. nltk-dependent  pipeline stages will not be loaded.



In [3]:
# | export
from pathlib import Path


def write(df: pl.DataFrame, fname: Path, format=None, **kwargs):
    if format is None:
        format = fname.suffix
        format = format[1:] if format.startswith(".") else format
    match format:
        case "arrow":
            df.write_ipc(fname, **kwargs)
        case "csv":
            df.write_csv(fname, **kwargs)
        case "parquet":
            df.write_parquet(fname, **kwargs)

    logger.info(f"Dataframe written to {fname}")

    return fname


class IdsEvents(BaseModel):
    """Core class to handle discontinuity events in a dataset."""

    class Config:
        extra = "allow"
        arbitrary_types_allowed = True

    name: str = None
    data: pl.LazyFrame = None
    ts: timedelta = None
    """time resolution of the dataset"""
    tau: timedelta = None
    """time interval used to find events"""
    events: pl.DataFrame = None
    method: Literal["fit", "derivative"] = "fit"

    @validate_call
    def export(self, path: Path, format="arrow", clean=True, **kwargs):
        if self.events is None:
            self.find_events()
        _df = self.events
        if clean:
            _df = _df.select(cs.datetime(), cs.duration(), cs.numeric())

        # check the parent directory exists
        path.parent.mkdir(parents=True, exist_ok=True)
        write(_df, path, format, **kwargs)
        return self

    def find_events(self, **kwargs):
        self.events = ids_finder(
            self.data, ts=self.ts, tau=self.tau, method=self.method, **kwargs
        )
        return self

    def get_event(self, index: int):
        events = self.events
        if "index" not in events.columns:
            events = events.with_row_index()
        predicate = pl.col("index") == index
        return events.row(by_predicate=predicate, named=True)

    def get_events(self, indices=None, predicate=None):
        pass

    def get_event_data(
        self,
        event,
        start_col="t.d_start",
        end_col="t.d_end",
        offset=timedelta(seconds=1),
        **kwargs,
    ):
        start = event[start_col] - offset
        end = event[end_col] + offset

        _data = self.data.filter(pl.col("time").is_between(start, end))
        return df2ts(_data, self.bcols)

In [ ]:
#| export
def log_event_change(event, logger=logger):
    logger.debug(
        f"""CHANGE INFO
        n.change: {event.get('n.change')}
        v.ion.change: {event.get('v.ion.change')}
        T.change: {event.get('T.change')}
        v.Alfven.change: {event.get('v.Alfven.change')}
        v.ion.change.l: {event.get('v.ion.change.l')}
        v.Alfven.change.l: {event.get('v.Alfven.change.l')}
        """
    )

In [ ]:
from fastcore.all import concat

ls = concat([None, [1], None])
ls.remove(None)
ls

In [ ]:
# def overview_plot(
#     self, event: dict, start=None, stop=None, offset=timedelta(seconds=1), **kwargs
# ):
#     # BUG: to be fixed
#     start = start or event["tstart"]
#     stop = stop or event["tstop"]

#     start -= offset
#     stop += offset

#     _plasma_data = self.plasma_data.filter(
#         pl.col("time").is_between(start, stop)
#     ).collect()

#     _mag_data = (
#         self.data.filter(pl.col("time").is_between(start, stop))
#         .collect()
#         .melt(
#             id_vars=["time"],
#             value_vars=self.bcols,
#             variable_name="B comp",
#             value_name="B",
#         )
#     )

#     v_df = _plasma_data.melt(
#         id_vars=["time"],
#         value_vars=self.plasma_meta.velocity_cols,
#         variable_name="veloity comp",
#         value_name="v",
#     )

#     panel_mag = _mag_data.hvplot(
#         x="time", y="B", by="B comp", ylabel="Magnetic Field", **kwargs
#     )
#     panel_n = _plasma_data.hvplot(
#         x="time", y=self.plasma_meta.density_col, **kwargs
#     ) * _plasma_data.hvplot.scatter(
#         x="time", y=self.plasma_meta.density_col, **kwargs
#     )

#     panel_v = v_df.hvplot(
#         x="time", y="v", by="veloity comp", ylabel="Plasma Velocity", **kwargs
#     )
#     panel_temp = _plasma_data.hvplot(
#         x="time", y=self.plasma_meta.temperature_col, **kwargs
#     )

#     mag_vlines = hv.VLine(event["t.d_start"]) * hv.VLine(event["t.d_end"])
#     plasma_vlines = hv.VLine(event.get("time_before")) * hv.VLine(
#         event.get("time_after")
#     )

#     logger.info(f"Overview plot: {event['tstart']} - {event['tstop']}")
#     log_event_change(event)

#     return (
#         panel_mag * mag_vlines
#         + panel_n * plasma_vlines
#         + panel_v * plasma_vlines
#         + panel_temp * plasma_vlines
#     ).cols(1)

In [ ]:
# | export
class IDsDataset(IdsEvents):
    """Extend the IdsEvents class to handle plasma and temperature data."""

    data: pl.LazyFrame = Field(default=None, alias="mag_data")
    mag_meta: Meta = Meta()
    bcols: list[str] = None

    plasma_data: pl.LazyFrame = None
    plasma_meta: PlasmaMeta = PlasmaMeta()

    ion_temp_data: pl.LazyFrame = None
    ion_temp_meta: TempMeta = TempMeta()
    e_temp_data: pl.LazyFrame = None
    e_temp_meta: TempMeta = TempMeta()

    def update_events(self, **kwargs):
        return self.update_events_with_plasma_data(
            **kwargs
        ).update_events_with_temp_data(**kwargs)

    def update_events_with_plasma_data(self, **kwargs):
        # TypeError: the truth value of a LazyFrame is ambiguous
        if self.plasma_data is not None:
            df_combined = combine_features(
                self.events,
                self.plasma_data.collect(),
                plasma_meta=self.plasma_meta,
                **kwargs,
            )

            self.events = calc_combined_features(
                df_combined,
                plasma_meta=self.plasma_meta,
                **kwargs,
            )
        else:
            logger.info("Plasma data is not available.")

        return self

    def update_events_with_temp_data(self, **kwargs):
        on = "time"

        if self.ion_temp_data is not None:
            self.events = self.events.sort(on).join_asof(
                self.ion_temp_data.sort(on).collect(), on=on
            )
        else:
            logger.info("Ion temperature data is not available.")

        if self.e_temp_data is not None:
            self.events = self.events.sort(on).join_asof(
                self.e_temp_data.sort(on).collect(), on=on
            )
        else:
            logger.info("Electron temperature data is not available.")
        return self

    def plot(self, type="overview", event=None, index=None, **kwargs):

        event = event or self.get_event(index)
        if type == "overview":
            return self.overview_plot(event, **kwargs)

    def plot_candidate(self, event=None, index=None, **kwargs):
        if event is None:
            event = self.get_event(index)
        data = self.get_event_data(event, **kwargs)

        return _plot_candidate(event, data, **kwargs)

    def plot_candidates(
        self, indices=None, num=4, random=True, predicate=None, **kwargs
    ):
        events = self.events
        if "index" not in events.columns:
            events = events.with_row_index()

        if indices is None:  # the truth value of an Expr is ambiguous
            if predicate is not None:
                events = events.filter(predicate)
            indices = events.get_column("index")
            if random:
                indices = indices.sample(num).to_numpy()
            else:
                indices = indices.head(num).to_numpy()
            logger.info(f"Candidates indices: {indices}")

        return [self.plot_candidate(index=i, **kwargs) for i in indices]

In [1]:
#| hide
from nbdev import nbdev_export
nbdev_export()